# Slice setup

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()

In [2]:
# from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
# fablib = fablib_manager()
# slice = fablib.get_slice(name="P4DPDK_HH")
slice = fablib.get_slice(name="P4DPDK_HH20")

# slice.delete()

In [3]:
slice.show();

ID,214f735b-7760-4efd-88c5-93c3c739836f
Name,P4DPDK_HH20
Lease Expiration (UTC),2025-04-20 19:03:34 +0000
Lease Start (UTC),2025-01-19 20:03:34 +0000
Project ID,8eaa3ec2-65e7-49a3-8c09-e1761141a6ad
State,StableOK


In [4]:
servers = []

servers.append(slice.get_node(name="server1"))     
servers.append(slice.get_node(name="server2"))
servers.append(slice.get_node(name="server3"))

server1 = servers[0]
server2 = servers[1]
server3 = servers[2]

## Reboot

In [6]:
for server in servers:
    server.os_reboot()

## Get interfaces names
In this step we will get the interface names so that we can assign IP addresses to them. Map the printed interface names to those seen in this figure:

<img src="./labs_files/SYN/figs/07_getting_interfaces.png" width="550"><br>

In [11]:
node1_iface = server1.get_interface(network_name='net1') 
server1_iface_name = node1_iface.get_device_name()#+'np0'
print(f'server1_iface: {server1_iface_name}')

node2_iface1 = server2.get_interface(network_name='net1') 
server2_iface1_name = node2_iface1.get_device_name()#+'np0'
print(f'server2_iface1: {server2_iface1_name}')

node2_iface2 = server2.get_interface(network_name='net2') 
server2_iface2_name = node2_iface2.get_device_name()#+'np1'
server2_iface2_name="enp8s0np1"
print(f'server2_iface2: {server2_iface2_name}')

node3_iface = server3.get_interface(network_name='net2') 
server3_iface_name = node3_iface.get_device_name()#+'np0'
print(f'server3_iface: {server3_iface_name}')

server1_iface: enp7s0np0
server2_iface1: enp7s0np0
server2_iface2: enp8s0np1
server3_iface: enp7s0np0


In [12]:
stdout, stderr = server1.execute(f'sudo ip link set dev {server1_iface_name} up', quiet=True)
stdout, stderr = server2.execute(f'sudo ip link set dev {server2_iface1_name} up', quiet=True)
stdout, stderr = server2.execute(f'sudo ip link set dev {server2_iface2_name} up', quiet=True)
stdout, stderr = server3.execute(f'sudo ip link set dev {server3_iface_name} up', quiet=True)

In [10]:
server1_iface_MAC = '00:00:00:00:00:01'
server2_iface1_MAC = '00:00:00:00:00:21'
server2_iface2_MAC = '00:00:00:00:00:22'
server3_iface_MAC = '00:00:00:00:00:03'

In [6]:
server1_server2_subnet = "192.168.10.0/24"
server1_ip = '192.168.10.1/24'
server2_1_ip = '192.168.10.2/24'

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} {server1_ip}')
stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface1_name} {server2_1_ip}')

stdout, stderr = server1.execute(f'sudo ifconfig {server1_iface_name} hw ether {server1_iface_MAC}')
stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface1_name} hw ether {server2_iface1_MAC}')

In [8]:
server2_server3_subnet = "192.168.20.0/24"
server2_2_ip = '192.168.20.2/24'
server3_ip = '192.168.20.1/24'

stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface2_name} {server2_2_ip}')
stdout, stderr = server3.execute(f'sudo ifconfig {server3_iface_name} {server3_ip}')

stdout, stderr = server2.execute(f'sudo ifconfig {server2_iface2_name} hw ether {server2_iface2_MAC}')
stdout, stderr = server3.execute(f'sudo ifconfig {server3_iface_name} hw ether {server3_iface_MAC}')

In [16]:
stdout, stderr = server2.execute(f'sudo sysctl -w net.ipv4.ip_forward=1', quiet=True)

In [11]:
ip1 = server1_ip.split('/')[0]
ip2_1 = server2_1_ip.split('/')[0]
ip2_2 = server2_2_ip.split('/')[0]
ip3 = server3_ip.split('/')[0]
subnet1 = "192.168.10.0/24"
subnet2 = "192.168.20.0/24"

stdout, stderr = server1.execute(f'sudo arp -s {ip2_1} {server2_iface1_MAC}')
stdout, stderr = server1.execute(f'sudo ip route add {subnet2} via {ip2_1}')

stdout, stderr = server2.execute(f'sudo arp -s {ip1} {server1_iface_MAC}')
stdout, stderr = server2.execute(f'sudo arp -s {ip3} {server3_iface_MAC}')

stdout, stderr = server3.execute(f'sudo arp -s {ip2_2} {server2_iface2_MAC}')
stdout, stderr = server3.execute(f'sudo ip route add {subnet1} via {ip2_2}')

 sudo: unable to resolve host server3: Name or service not known
  sudo: unable to resolve host server3: Name or service not known
 

In [18]:
stdout, stderr = server1.execute(f'sudo ibdev2netdev')
stdout, stderr = server1.execute(f'sudo mst status', quiet=True)
stdout, stderr = server1.execute(f'sudo mst start', quiet=True)
stdout, stderr = server1.execute(f'sudo mst status', quiet=True)

stdout, stderr = server2.execute(f'sudo ibdev2netdev')
stdout, stderr = server2.execute(f'sudo mst status', quiet=True)
stdout, stderr = server2.execute(f'sudo mst start', quiet=True)
stdout, stderr = server2.execute(f'sudo mst status', quiet=True)

stdout, stderr = server3.execute(f'sudo ibdev2netdev')
stdout, stderr = server3.execute(f'sudo mst status', quiet=True)
stdout, stderr = server3.execute(f'sudo mst start', quiet=True)
stdout, stderr = server3.execute(f'sudo mst status', quiet=True)

mlx5_0 port 1 ==> enp7s0np0 (Up)
mlx5_1 port 1 ==> enp8s0np1 (Down)
 sudo: unable to resolve host server1: Name or service not known
 mlx5_0 port 1 ==> enp7s0np0 (Up)
mlx5_1 port 1 ==> enp8s0np1 (Up)
 sudo: unable to resolve host server2: Name or service not known
 mlx5_0 port 1 ==> enp7s0np0 (Up)
mlx5_1 port 1 ==> enp8s0np1 (Down)
 sudo: unable to resolve host server3: Name or service not known
 

# Get SSH

In [3]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe7e:26b2'

In [4]:
server2.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe22:a10'

In [5]:
server3.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe65:a42d'

# Upload and Compile

In [6]:
threads = []

for server in servers:
    threads.append(server.execute_thread(f' sudo sh -c  "echo 1024 > /sys/kernel/mm/hugepages/hugepages-2048kB/nr_hugepages"'))

for thread in threads:
    thread.result()

## Reflector

In [12]:
server2.upload_file('experiments/reflector/reflector.p4','reflector.p4')
stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna reflector.p4 -o reflector.spec')

 sudo: unable to resolve host server2: Name or service not known
 

In [16]:
server2.upload_file('experiments/reflector/ethdev0.io','ethdev0.io')
server2.upload_file('experiments/reflector/reflector.cli','reflector.cli')

<SFTPAttributes: [ size=556 uid=1000 gid=1000 mode=0o100664 atime=1729096560 mtime=1729096560 ]>

    cd dpdk
    sudo examples/pipeline/build/pipeline -c 0x1F -- -s /home/ubuntu/reflector.cli

In [17]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:18e8:fff0:3:f816:3eff:fe79:77b8'

In [57]:
stdout, stderr = server1.execute('lspci | grep ConnectX | awk \'{print $1}\'| head -n 1')

07:00.0


    sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"

    set 0 src mac 00:00:00:00:00:01
    set 0 dst mac 00:00:00:00:00:21
    set 0 src ip 192.168.10.1/24
    set 0 dst ip 192.168.10.2
    set 0 size 1500

In [18]:
server3.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe8d:7886'

In [31]:
stdout, stderr = server3.execute('lspci | grep ConnectX | awk \'{print $1}\'| head -n 1')

07:00.0


    sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"

    set 0 src mac 00:00:00:00:00:03
    set 0 dst mac 00:00:00:00:00:22
    set 0 src ip 192.168.20.1/24
    set 0 dst ip 192.168.20.2
    set 0 size 1500

## Heavy Hitter

In [29]:
server2.upload_file('experiments/hh/hh.p4','hh.p4')
stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna hh.p4 -o hh.spec')

 sudo: unable to resolve host server2: Name or service not known
 

In [8]:
server2.upload_file('experiments/hh/ethdev0.io','ethdev0.io')
server2.upload_file('experiments/hh/ethdev1.io','ethdev1.io')
server2.upload_file('experiments/hh/ethdev2.io','ethdev2.io')
server2.upload_file('experiments/hh/ethdev3.io','ethdev3.io')
server2.upload_file('experiments/hh/hh.cli','hh.cli')

<SFTPAttributes: [ size=907 uid=1000 gid=1000 mode=0o100664 atime=1729192859 mtime=1729192860 ]>

    cd dpdk
    sudo examples/pipeline/build/pipeline -c 0x1F -- -s /home/ubuntu/hh.cli

In [9]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fe4f:3067'

In [10]:
stdout, stderr = server1.execute('lspci | grep ConnectX | awk \'{print $1}\'| head -n 1')

07:00.0


    sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"

    page range

    range 0 src mac start 00:00:00:00:00:01
    range 0 src mac min 00:00:00:00:00:01
    range 0 src mac max 00:00:00:00:00:01
    
    range 0 dst mac start 00:00:00:00:00:21
    range 0 dst mac min 00:00:00:00:00:21
    range 0 dst mac max 00:00:00:00:00:21
    
    range 0 src ip start 1.1.1.1
    range 0 src ip min 1.1.1.1
    range 0 src ip max 250.250.250.250
    range 0 src ip inc 0.0.0.1
    
    range 0 dst ip start 1.1.1.1
    range 0 dst ip min 1.1.1.1
    range 0 dst ip max 250.250.250.250
    range 0 dst ip inc 0.0.0.1
    
    range 0 size start 1500
    range 0 size min 1500
    range 0 size max 1500
    range 0 size inc 0
    
    enable 0 range
    page main

In [11]:
server3.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fef9:8841'

In [12]:
stdout, stderr = server3.execute('lspci | grep ConnectX | awk \'{print $1}\'| head -n 1')

07:00.0


    sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"

    set 0 src mac 00:00:00:00:00:03
    set 0 dst mac 00:00:00:00:00:22
    set 0 src ip 192.168.20.1/24
    set 0 dst ip 192.168.20.2
    set 0 size 1500
    set 0 rate 1

## SYN Flood

In [13]:
server2.upload_file('experiments/syn/syn.p4','syn.p4')
stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna syn.p4 -o syn.spec')

 sudo: unable to resolve host server2: Name or service not known
 

In [14]:
server2.upload_file('experiments/syn/ethdev0.io','ethdev0.io')
server2.upload_file('experiments/syn/ethdev1.io','ethdev1.io')
server2.upload_file('experiments/syn/ethdev2.io','ethdev2.io')
server2.upload_file('experiments/syn/ethdev3.io','ethdev3.io')
server2.upload_file('experiments/syn/syn.cli','syn.cli')

server2.upload_file('experiments/syn/rules.txt','rules.txt')
server2.upload_file('experiments/syn/reset_SYN_packets_per_second.py','reset_SYN_packets_per_second.py')

<SFTPAttributes: [ size=1183 uid=1000 gid=1000 mode=0o100664 atime=1729194058 mtime=1729194058 ]>

In [15]:
server2.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fe30:337f'

    cd dpdk
    sudo examples/pipeline/build/pipeline -c 0x1F -- -s /home/ubuntu/syn.cli

in a new terminal:
    
    sudo python3 reset_SYN_packets_per_second.py

In [9]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fe4f:3067'

In [10]:
stdout, stderr = server1.execute('lspci | grep ConnectX | awk \'{print $1}\'| head -n 1')

07:00.0


    sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"

    page range
    
    range 0 src mac start 00:00:00:00:00:01
    range 0 src mac min 00:00:00:00:00:01
    range 0 src mac max 00:00:00:00:00:01
    
    range 0 dst mac start 00:00:00:00:00:21
    range 0 dst mac min 00:00:00:00:00:21
    range 0 dst mac max 00:00:00:00:00:21
    
    range 0 src ip start 1.1.1.1
    range 0 src ip min 1.1.1.1
    range 0 src ip max 250.250.250.250
    range 0 src ip inc 0.0.0.1
    
    range 0 dst ip start 192.168.20.1
    range 0 dst ip min 192.168.20.1
    range 0 dst ip max 192.168.20.1
    
    range 0 size start 1500
    range 0 size min 1500
    range 0 size max 1500
    
    range 0 tcp flag clr all
    range 0 tcp flag set syn
    
    enable 0 range
    page main

In [11]:
server3.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fef9:8841'

In [12]:
stdout, stderr = server3.execute('lspci | grep ConnectX | awk \'{print $1}\'| head -n 1')

07:00.0


    sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"

In [16]:
server2.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fe30:337f'

    telnet 0.0.0.0 8086

    pipeline PIPELINE0 regwr drop_percent_reg_0 value 50 index 0
    pipeline PIPELINE1 regwr drop_percent_reg_0 value 50 index 0
    pipeline PIPELINE2 regwr drop_percent_reg_0 value 50 index 0
    pipeline PIPELINE3 regwr drop_percent_reg_0 value 50 index 0
    
    pipeline PIPELINE0 regwr drop_percent_reg_0 value 100 index 0
    pipeline PIPELINE1 regwr drop_percent_reg_0 value 100 index 0
    pipeline PIPELINE2 regwr drop_percent_reg_0 value 100 index 0
    pipeline PIPELINE3 regwr drop_percent_reg_0 value 100 index 0

## Interarrival Time

In [16]:
# stdout, stderr = server2.execute(f'cd dpdk/lib/pipeline/ &&  sudo rm rte_swx_pipeline.c && sudo rm rte_swx_pipeline_internal.h', quiet = True)
# server2.upload_file('scripts/rte_swx_pipeline.c','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline.c')
# server2.upload_file('scripts/rte_swx_pipeline_internal.h','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline_internal.h')
# stdout, stderr = server2.execute(f'cd dpdk && cd build && sudo ninja && sudo ninja install && sudo ldconfig', quiet = True)

In [24]:
server2.upload_file('experiments/iat/iat.p4','iat.p4')
stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna iat.p4 -o iat.spec')

 sudo: unable to resolve host server2: Name or service not known
 

In [25]:
server2.download_file('experiments/iat/iat.spec','iat.spec')

In [26]:
server2.upload_file('experiments/iat/ethdev0.io','ethdev0.io')
# server2.upload_file('experiments/iat/ethdev1.io','ethdev1.io')
# server2.upload_file('experiments/iat/ethdev2.io','ethdev2.io')
# server2.upload_file('experiments/iat/ethdev3.io','ethdev3.io')
server2.upload_file('experiments/iat/iat.cli','iat.cli')

server2.upload_file('experiments/iat/rules.txt','rules.txt')

<SFTPAttributes: [ size=112 uid=1000 gid=1000 mode=0o100664 atime=1729783507 mtime=1729809095 ]>

In [27]:
server2.upload_file('experiments/iat/get_iat.py','get_iat.py')

<SFTPAttributes: [ size=3128 uid=1000 gid=1000 mode=0o100664 atime=1729803637 mtime=1729809102 ]>

    cd dpdk
    sudo examples/pipeline/build/pipeline -c 0x1F -- -s /home/ubuntu/iat.cli
    sudo python3 get_iat.py 

In [28]:
server3.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fef9:8841'

    sudo ping 192.168.20.2 -i 1

## Heavy Hitter and SYN Flood

In [29]:
server2.upload_file('experiments/hh_syn/syn_flood.p4','syn_flood.p4')
server2.upload_file('experiments/hh_syn/hh_syn.p4','hh_syn.p4')
server2.upload_file('experiments/hh_syn/registers.p4','registers.p4')
server2.upload_file('experiments/hh_syn/headers.p4','headers.p4')
stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna hh_syn.p4 -o hh_syn.spec')

FileNotFoundError: [Errno 2] No such file or directory: 'experiments/hh_syn/registers.p4'

In [34]:
server2.upload_file('experiments/hh_syn/hh_syn.p4','hh_syn.p4')
stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna hh_syn.p4 -o hh_syn.spec')

 sudo: unable to resolve host server2: Name or service not known
 

In [31]:
server2.upload_file('experiments/hh_syn/ethdev0.io','ethdev0.io')
server2.upload_file('experiments/hh_syn/ethdev1.io','ethdev1.io')
server2.upload_file('experiments/hh_syn/ethdev2.io','ethdev2.io')
server2.upload_file('experiments/hh_syn/ethdev3.io','ethdev3.io')
server2.upload_file('experiments/hh_syn/hh_syn.cli','hh_syn.cli')

server2.upload_file('experiments/hh_syn/rules.txt','rules.txt')

<SFTPAttributes: [ size=112 uid=1000 gid=1000 mode=0o100664 atime=1729809127 mtime=1729864763 ]>

    cd dpdk
    sudo examples/pipeline/build/pipeline -c 0x1F -- -s /home/ubuntu/hh_syn.cli

In [32]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fe4f:3067'

In [10]:
stdout, stderr = server1.execute('lspci | grep ConnectX | awk \'{print $1}\'| head -n 1')

07:00.0


    sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"

    page range

    range 0 src mac start 00:00:00:00:00:01
    range 0 src mac min 00:00:00:00:00:01
    range 0 src mac max 00:00:00:00:00:01
    
    range 0 dst mac start 00:00:00:00:00:21
    range 0 dst mac min 00:00:00:00:00:21
    range 0 dst mac max 00:00:00:00:00:21
    
    range 0 src ip start 1.1.1.1
    range 0 src ip min 1.1.1.1
    range 0 src ip max 250.250.250.250
    range 0 src ip inc 0.0.0.1
    
    range 0 dst ip start 192.168.20.1
    range 0 dst ip min 192.168.20.1
    range 0 dst ip max 192.168.20.1
    
    range 0 size start 1500
    range 0 size min 1500
    range 0 size max 1500

    range 0 tcp flag clr all
    range 0 tcp flag set syn
    
    enable 0 range
    page main
    

In [33]:
server3.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fef9:8841'

In [12]:
stdout, stderr = server3.execute('lspci | grep ConnectX | awk \'{print $1}\'| head -n 1')

07:00.0


    sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"

    set 0 src mac 00:00:00:00:00:03
    set 0 dst mac 00:00:00:00:00:22
    set 0 src ip 192.168.20.1/24
    set 0 dst ip 192.168.10.1
    set 0 size 1500
    set 0 rate 0.5

## SYN-ACK Flood

In [18]:
server2.upload_file('experiments/syn_ack/syn_ack.p4','syn_ack.p4')
stdout, stderr = server2.execute(f'sudo p4c-dpdk --arch=pna syn_ack.p4 -o syn_ack.spec')

 sudo: unable to resolve host server2: Name or service not known
 

In [7]:
server2.upload_file('experiments/syn_ack/ethdev0.io','ethdev0.io')
server2.upload_file('experiments/syn_ack/ethdev1.io','ethdev1.io')
server2.upload_file('experiments/syn_ack/ethdev2.io','ethdev2.io')
server2.upload_file('experiments/syn_ack/ethdev3.io','ethdev3.io')
server2.upload_file('experiments/syn_ack/syn_ack.cli','syn_ack.cli')

server2.upload_file('experiments/syn_ack/rules.txt','rules.txt')

<SFTPAttributes: [ size=112 uid=1000 gid=1000 mode=0o100664 atime=1730129558 mtime=1730129765 ]>

In [15]:
server2.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fe30:337f'

    cd dpdk
    sudo examples/pipeline/build/pipeline -c 0x1F -- -s /home/ubuntu/syn_ack.cli

In [9]:
server1.get_ssh_command()

'ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3080:f816:3eff:fe4f:3067'

In [10]:
stdout, stderr = server1.execute('lspci | grep ConnectX | awk \'{print $1}\'| head -n 1')

07:00.0


    sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"
    
    page range

    range 0 src mac start 00:00:00:00:00:01
    range 0 src mac min 00:00:00:00:00:01
    range 0 src mac max 00:00:00:00:00:01
    
    range 0 dst mac start 00:00:00:00:00:21
    range 0 dst mac min 00:00:00:00:00:21
    range 0 dst mac max 00:00:00:00:00:21
    
    range 0 src ip start 1.1.1.1
    range 0 src ip min 1.1.1.1
    range 0 src ip max 250.250.250.250
    range 0 src ip inc 0.0.0.1
    
    range 0 dst ip start 192.168.20.1
    range 0 dst ip min 192.168.20.1
    range 0 dst ip max 192.168.20.1
    
    range 0 size start 1500
    range 0 size min 1500
    range 0 size max 1500
    
    range 0 tcp flag clr all
    range 0 tcp flag set syn
    range 0 tcp flag set ack
    
    enable 0 range
    page main

## CMS

In [14]:
server2.upload_file('main/scripts/cms_optimal.p4','main/cms_optimal.p4')
stdout, stderr = server2.execute(f'cd main && sudo p4c-dpdk --arch=pna cms_optimal.p4 -o cms_optimal.spec')

 sudo: unable to resolve host server2: Name or service not known
 

## MAIN

In [7]:
stdout, stderr = server2.execute(f'mkdir main')

In [67]:
# server2.upload_file('main/ack_flood.p4','main/ack_flood.p4')
server2.upload_file('main/control.p4','main/control.p4')
# server2.upload_file('main/deparser.p4','main/deparser.p4')
# server2.upload_file('main/fin_flood.p4','main/fin_flood.p4')
# server2.upload_file('main/get_icmp_flow.p4','main/get_icmp_flow.p4')
# server2.upload_file('main/get_tcp_flow.p4','main/get_tcp_flow.p4')
server2.upload_file('main/headers.p4','main/headers.p4')
# server2.upload_file('main/heavy_hitter.p4','main/heavy_hitter.p4')
# server2.upload_file('main/icmp_flood.p4','main/icmp_flood.p4')
# server2.upload_file('main/main.p4','main/main.p4')
# server2.upload_file('main/parser.p4','main/parser.p4')
# server2.upload_file('main/registers.p4','main/registers.p4')
# server2.upload_file('main/syn_ack_flood.p4','main/syn_ack_flood.p4')
# server2.upload_file('main/syn_flood.p4','main/syn_flood.p4')
server2.upload_file('main/udp_flood.p4','main/udp_flood.p4')
stdout, stderr = server2.execute(f'cd main && sudo p4c-dpdk --arch=pna main.p4 -o main.spec')

 sudo: unable to resolve host server2: Name or service not known
registers.p4(6): [--Wwarn=unused] warning: reg_state_ts: unused instance
    Register<bit<2>, bit<1>>(1) reg_state_ts;
                                ^^^^^^^^^^^^
registers.p4(7): [--Wwarn=unused] warning: reg_timestamp1: unused instance
    Register<bit<32>, bit<1>>(1) reg_timestamp1;
                                 ^^^^^^^^^^^^^^
registers.p4(8): [--Wwarn=unused] warning: reg_timestamp2: unused instance
    Register<bit<32>, bit<1>>(1) reg_timestamp2;
                                 ^^^^^^^^^^^^^^
registers.p4(9): [--Wwarn=unused] warning: reg_timestamp3: unused instance
    Register<bit<32>, bit<1>>(1) reg_timestamp3;
                                 ^^^^^^^^^^^^^^
 

In [8]:
server2.upload_file('main_sec/ack_flood.p4','main/ack_flood.p4')
server2.upload_file('main_sec/control.p4','main/control.p4')
server2.upload_file('main_sec/deparser.p4','main/deparser.p4')
server2.upload_file('main_sec/fin_flood.p4','main/fin_flood.p4')
server2.upload_file('main_sec/get_icmp_flow.p4','main/get_icmp_flow.p4')
server2.upload_file('main_sec/get_tcp_flow.p4','main/get_tcp_flow.p4')
server2.upload_file('main_sec/headers.p4','main/headers.p4')
server2.upload_file('main_sec/heavy_hitter.p4','main/heavy_hitter.p4')
server2.upload_file('main_sec/icmp_flood.p4','main/icmp_flood.p4')
server2.upload_file('main_sec/main.p4','main/main.p4')
server2.upload_file('main_sec/parser.p4','main/parser.p4')
server2.upload_file('main_sec/registers.p4','main/registers.p4')
server2.upload_file('main_sec/syn_ack_flood.p4','main/syn_ack_flood.p4')
server2.upload_file('main_sec/syn_flood.p4','main/syn_flood.p4')
server2.upload_file('main_sec/udp_flood.p4','main/udp_flood.p4')
stdout, stderr = server2.execute(f'cd main && sudo p4c-dpdk --arch=pna main.p4 -o main.spec')

 sudo: unable to resolve host server2: Name or service not known
registers.p4(6): [--Wwarn=unused] warning: reg_state_ts: unused instance
    Register<bit<2>, bit<1>>(1) reg_state_ts;
                                ^^^^^^^^^^^^
registers.p4(7): [--Wwarn=unused] warning: reg_timestamp1: unused instance
    Register<bit<32>, bit<1>>(1) reg_timestamp1;
                                 ^^^^^^^^^^^^^^
registers.p4(8): [--Wwarn=unused] warning: reg_timestamp2: unused instance
    Register<bit<32>, bit<1>>(1) reg_timestamp2;
                                 ^^^^^^^^^^^^^^
registers.p4(9): [--Wwarn=unused] warning: reg_timestamp3: unused instance
    Register<bit<32>, bit<1>>(1) reg_timestamp3;
                                 ^^^^^^^^^^^^^^
 

In [9]:
server2.upload_file('main/scripts/rules.txt','main/rules.txt')
server2.upload_file('main/scripts/ethdev0.io','main/ethdev0.io')
server2.upload_file('main/scripts/ethdev1.io','main/ethdev1.io')
server2.upload_file('main/scripts/ethdev2.io','main/ethdev2.io')
server2.upload_file('main/scripts/ethdev3.io','main/ethdev3.io')
server2.upload_file('main/scripts/main.cli','main/main.cli')

<SFTPAttributes: [ size=1349 uid=1000 gid=1000 mode=0o100664 atime=1737322072 mtime=1737322072 ]>

    cd dpdk
    sudo examples/pipeline/build/pipeline -c 0x1F -- -s /home/ubuntu/main/main.cli

In [9]:
# server2.upload_file('main/scripts/telnet.py','main/telnet.py')
# server2.upload_file('main/scripts/print_flow.py','main/print_flow.py')
# server2.upload_file('main/scripts/clear.py','main/clear.py')

<SFTPAttributes: [ size=837 uid=1000 gid=1000 mode=0o100664 atime=1735932697 mtime=1735932697 ]>

In [4]:
# server1.upload_file('main/scripts/ethtool.sh','ethtool.sh')
# stdout, stderr = server1.execute(f'sudo chmod +x ethtool.sh')
# server2.upload_file('main/scripts/ethtool.sh','ethtool.sh')
# stdout, stderr = server2.execute(f'sudo chmod +x ethtool.sh')
# server3.upload_file('main/scripts/ethtool.sh','ethtool.sh')
# stdout, stderr = server3.execute(f'sudo chmod +x ethtool.sh')
# server2.upload_file('main/scripts/srcip.sh','srcip.sh')
# stdout, stderr = server2.execute(f'sudo chmod +x srcip.sh')

 sudo: unable to resolve host server2: Name or service not known
 

In [19]:
# server3.upload_file('main/scripts/send.py','send.py')
# stdout, stderr = server3.execute(f'sudo chmod +x send.py')

 sudo: unable to resolve host server3: Name or service not known
 

In [18]:
server2.download_file('main/main.spec','main/main.spec')

In [8]:
# server2.download_file('src_ips.csv','src_ips.csv')

In [6]:
# server3.download_file('network_packets.csv','network_packets.csv')

### MULTICORE

In [10]:
stdout, stderr = server2.execute(f'mkdir multicore')

In [4]:
server2.upload_file('main/multicore/rules.txt','multicore/rules.txt')
server2.upload_file('main/multicore/ethdev0.io','multicore/ethdev0.io')
server2.upload_file('main/multicore/ethdev1.io','multicore/ethdev1.io')
server2.upload_file('main/multicore/ethdev2.io','multicore/ethdev2.io')
server2.upload_file('main/multicore/ethdev3.io','multicore/ethdev3.io')
server2.upload_file('main/multicore/ethdev4.io','multicore/ethdev4.io')
server2.upload_file('main/multicore/ethdev5.io','multicore/ethdev5.io')
server2.upload_file('main/multicore/ethdev6.io','multicore/ethdev6.io')
server2.upload_file('main/multicore/ethdev7.io','multicore/ethdev7.io')
server2.upload_file('main/multicore/p0.cli','multicore/p0.cli')
server2.upload_file('main/multicore/p1.cli','multicore/p1.cli')
server2.upload_file('main/multicore/p2.cli','multicore/p2.cli')
server2.upload_file('main/multicore/p3.cli','multicore/p3.cli')
server2.upload_file('main/multicore/p4.cli','multicore/p4.cli')
server2.upload_file('main/multicore/p5.cli','multicore/p5.cli')
server2.upload_file('main/multicore/p6.cli','multicore/p6.cli')
server2.upload_file('main/multicore/p7.cli','multicore/p7.cli')

<SFTPAttributes: [ size=14617 uid=1000 gid=1000 mode=0o100664 atime=1737322400 mtime=1738182061 ]>

sudo examples/pipeline/build/pipeline -c 0x03 -- -s /home/ubuntu/multicore/p0.cli
sudo examples/pipeline/build/pipeline -c 0x07 -- -s /home/ubuntu/multicore/p1.cli
sudo examples/pipeline/build/pipeline -c 0x0F -- -s /home/ubuntu/multicore/p2.cli
sudo examples/pipeline/build/pipeline -c 0x1F -- -s /home/ubuntu/multicore/p3.cli
sudo examples/pipeline/build/pipeline -c 0x3F -- -s /home/ubuntu/multicore/p4.cli
sudo examples/pipeline/build/pipeline -c 0x7F -- -s /home/ubuntu/multicore/p5.cli
sudo examples/pipeline/build/pipeline -c 0xFF -- -s /home/ubuntu/multicore/p6.cli
sudo examples/pipeline/build/pipeline -c 0x1FF -- -s /home/ubuntu/multicore/p7.cli

### Timestamps

In [20]:
# stdout, stderr = server2.execute(f'cd dpdk/lib/pipeline/ &&  sudo rm rte_swx_pipeline.c && sudo rm rte_swx_pipeline_internal.h', quiet = True)
# server2.upload_file('scripts/rte_swx_pipeline.c','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline.c')
# server2.upload_file('scripts/rte_swx_pipeline_internal.h','/home/ubuntu/dpdk/lib/pipeline/rte_swx_pipeline_internal.h')
# stdout, stderr = server2.execute(f'cd dpdk && cd build && sudo ninja && sudo ninja install && sudo ldconfig', quiet = True)

In [23]:
server2.upload_file('main/multicore/telnet0.py','multicore/telnet0.py')
server2.upload_file('main/multicore/telnet1.py','multicore/telnet1.py')
server2.upload_file('main/multicore/telnet2.py','multicore/telnet2.py')
server2.upload_file('main/multicore/telnet3.py','multicore/telnet3.py')
server2.upload_file('main/multicore/telnet4.py','multicore/telnet4.py')
server2.upload_file('main/multicore/telnet5.py','multicore/telnet5.py')
server2.upload_file('main/multicore/telnet6.py','multicore/telnet6.py')
server2.upload_file('main/multicore/telnet7.py','multicore/telnet7.py')

<SFTPAttributes: [ size=15263 uid=1000 gid=1000 mode=0o100664 atime=1737322400 mtime=1738190363 ]>

### SURICATA DPDK

In [13]:
stdout, stderr = server2.execute(f'mkdir myrules')

In [38]:
# server2.upload_file('main/suricata/syn.rules','myrules/syn.rules')
server2.upload_file('main/suricata/synack.rules','myrules/synack.rules')
# # server2.upload_file('main/suricata/ack.rules','myrules/ack.rules')
# server2.upload_file('main/suricata/fin.rules','myrules/fin.rules')
# server2.upload_file('main/suricata/hh.rules','myrules/hh.rules')
# server2.upload_file('main/suricata/icmp.rules','myrules/icmp.rules')
# server2.upload_file('main/suricata/udp.rules','myrules/udp.rules')
stdout, stderr = server2.execute(f'sudo cp ~/myrules/*.rules /usr/local/var/lib/suricata/rules/')

 sudo: unable to resolve host server2: Name or service not known
 

In [ ]:
stdout, stderr = server3.execute(f'mkdir suricata')

In [5]:
# server3.upload_file('main/suricata/syn.py','suricata/syn.py')
server3.upload_file('main/suricata/synack.py','suricata/synack.py')
# # server3.upload_file('main/suricata/ack.py','suricata/ack.py')
# server3.upload_file('main/suricata/fin.py','suricata/fin.py')
# server3.upload_file('main/suricata/hh.py','suricata/hh.py')
# server3.upload_file('main/suricata/icmp.py','suricata/icmp.py')
# server3.upload_file('main/suricata/udp.py','suricata/udp.py')

<SFTPAttributes: [ size=1109 uid=1000 gid=1000 mode=0o100664 atime=1737856358 mtime=1737856747 ]>

In [40]:
stdout, stderr = server2.execute(f'sudo cp /usr/local/etc/suricata/suricata.yaml ./')
stdout, stderr = server2.execute(f' sudo chmod o+r suricata.yaml')
server2.download_file('main/suricata/suricata.yaml','suricata.yaml')

 sudo: unable to resolve host server2: Name or service not known
  sudo: unable to resolve host server2: Name or service not known
 

In [41]:
server2.upload_file('main/suricata/suricata.yaml','myrules/suricata.yaml')
stdout, stderr = server2.execute(f'sudo cp ~/myrules/suricata.yaml /usr/local/etc/suricata/')

 sudo: unable to resolve host server2: Name or service not known
 

In [40]:
stdout, stderr = server2.execute(f'sudo pkill -f -9 suricata')

 sudo: unable to resolve host server2: Name or service not known
 

sudo cd /usr/local/var/lib/suricata/rules/

sudo vi /usr/local/etc/suricata/suricata.yaml 

sudo suricata --dpdk -vvvv -c /usr/local/etc/suricata/suricata.yaml

sudo cat /usr/local/var/log/suricata/stats.log 
echo "" > /usr/local/var/log/suricata/fast.log
cat /usr/local/var/log/suricata/fast.log | tail -n 10

sudo pktgen -l 0,1 -n 4 -a 07:00.0 -- -P -m "1.0"

### AOM

In [11]:
server2.upload_file('main/aom/control.p4','main/control.p4')
server2.upload_file('main/aom/deparser.p4','main/deparser.p4')
# server2.upload_file('main/aom/fin_flood.p4','main/fin_flood.p4')
server2.upload_file('main/aom/headers.p4','main/headers.p4')
server2.upload_file('main/aom/main.p4','main/main.p4')
server2.upload_file('main/aom/parser.p4','main/parser.p4')
# server2.upload_file('main/aom/registers.p4','main/registers.p4')
# server2.upload_file('main/aom/syn_flood.p4','main/syn_flood.p4')
stdout, stderr = server2.execute(f'cd main && sudo p4c-dpdk --arch=pna main.p4 -o main.spec')

 sudo: unable to resolve host server2: Name or service not known
 

## MAIN EXTENDED

In [7]:
stdout, stderr = server2.execute(f'mkdir main')

In [3]:
# server2.upload_file('main/ack_flood.p4','main/ack_flood.p4')
# server2.upload_file('main/control.p4','main/control.p4')
# server2.upload_file('main/deparser.p4','main/deparser.p4')
# server2.upload_file('main/fin_flood.p4','main/fin_flood.p4')
# server2.upload_file('main/get_icmp_flow.p4','main/get_icmp_flow.p4')
# server2.upload_file('main/get_tcp_flow.p4','main/get_tcp_flow.p4')
# server2.upload_file('main/headers.p4','main/headers.p4')
# server2.upload_file('main/heavy_hitter.p4','main/heavy_hitter.p4')
# server2.upload_file('main/icmp_flood.p4','main/icmp_flood.p4')
# server2.upload_file('main/main.p4','main/main.p4')
# server2.upload_file('main/parser.p4','main/parser.p4')
# server2.upload_file('main/registers.p4','main/registers.p4')
# server2.upload_file('main/syn_ack_flood.p4','main/syn_ack_flood.p4')
# server2.upload_file('main/syn_flood.p4','main/syn_flood.p4')
# server2.upload_file('main/udp_flood.p4','main/udp_flood.p4')
# stdout, stderr = server2.execute(f'cd main && sudo p4c-dpdk --arch=pna main.p4 -o main.spec')

 sudo: unable to resolve host server2: Name or service not known
 

In [9]:
server2.upload_file('main/control.p4','main/control.p4')
server2.upload_file('main/deparser.p4','main/deparser.p4')
server2.upload_file('main/headers.p4','main/headers.p4')
server2.upload_file('main/main.p4','main/main.p4')
server2.upload_file('main/parser.p4','main/parser.p4')
server2.upload_file('main/registers.p4','main/registers.p4')
server2.upload_file('main/syn_flood.p4','main/syn_flood.p4')
stdout, stderr = server2.execute(f'cd main && sudo p4c-dpdk --arch=pna main.p4 -o main.spec')

 sudo: unable to resolve host server2: Name or service not known
 

In [5]:
server2.upload_file('main/scripts/rules.txt','main/rules.txt')
server2.upload_file('main/scripts/ethdev0.io','main/ethdev0.io')
server2.upload_file('main/scripts/ethdev1.io','main/ethdev1.io')
server2.upload_file('main/scripts/ethdev2.io','main/ethdev2.io')
server2.upload_file('main/scripts/ethdev3.io','main/ethdev3.io')
server2.upload_file('main/scripts/main.cli','main/main.cli')

<SFTPAttributes: [ size=1349 uid=1000 gid=1000 mode=0o100664 atime=1737322113 mtime=1739845695 ]>

    cd dpdk
    sudo examples/pipeline/build/pipeline -c 0x1F -- -s /home/ubuntu/main/main.cli

In [25]:
server2.upload_file('main/scripts/threshold.py','main/threshold.py')
# server2.upload_file('main/scripts/getpps.py','main/getpps.py')
# server2.upload_file('main/scripts/write.py','main/write.py')

<SFTPAttributes: [ size=6000 uid=1000 gid=1000 mode=0o100664 atime=1744043382 mtime=1744044662 ]>

In [26]:
server2.download_file('main/scripts/output.txt','main/output.txt')

In [8]:
server1.upload_file('main/scripts/ethtool.sh','ethtool.sh')
stdout, stderr = server1.execute(f'sudo chmod +x ethtool.sh')

server3.upload_file('main/scripts/ethtool.sh','ethtool.sh')
stdout, stderr = server3.execute(f'sudo chmod +x ethtool.sh')

 sudo: unable to resolve host server1: Name or service not known
  sudo: unable to resolve host server3: Name or service not known
 

In [37]:
server3.download_file('network_packets.csv','network_packets.csv')

# MoonGen

In [ ]:
#https://scholzd.github.io/MoonGen/download.html
#https://github.com/emmericp/MoonGen?tab=readme-ov-file
stdout, stderr = server3.execute('sudo sudo apt install libtbb2', quiet = True)
stdout, stderr = server3.execute('git clone https://github.com/emmericp/MoonGen.git', quiet = True)
stdout, stderr = server3.execute('cd p4c && ./build.sh && ./setup-hugetlbfs.sh', quiet = True)
#sudo ./build/MoonGen ./examples/l3-load-latency.lua 0 1

# Step 10: Delete the slice

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
slice = fablib.get_slice(name="P4DPDK_HH4")
# slice.delete()